# Residential Power Usage Prediction

In [ ]:
import os
import zipfile
import numpy as np
import pandas as pd
import seaborn as sns
from google.colab import drive
import matplotlib.pyplot as plt

#! pip install cloudmesh-common -U
from cloudmesh.common.StopWatch import StopWatch

# Misc
import warnings
warnings.filterwarnings('ignore')

In [ ]:
drive.mount('/content/drive', force_remount=True)


In [ ]:
%cd /content/drive/My Drive/BigData_Project/Data

## Download datasets

In [ ]:
#!pip install kaggle

In [ ]:
if not os.path.exists('residential-power-usage-3years-data-timeseries.zip'):
  StopWatch.start("Data download")
  os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/My Drive/BigData_Project" # put path for wherever you put it
  !mkdir ~/.kaggle
  !cp /content/drive/'My Drive'/BigData_Project/kaggle.json ~/.kaggle
  !chmod 600 ~/.kaggle/kaggle.json
  !kaggle datasets download -d srinuti/residential-power-usage-3years-data-timeseries
  StopWatch.stop("Data download")
  StopWatch.status("Data download", True)

  0% 0.00/227k [00:00<?, ?B/s]
100% 227k/227k [00:00<00:00, 31.0MB/s]


## Load datasets

In [ ]:
data_path = os.getcwd()

In [ ]:

unzippingReq = True #please modify this flag unzippingReq from False to True 
if unzippingReq: #please modify this code unzippingReq from False to True 
    StopWatch.start("Data load")
    file_name = data_path  + '/residential-power-usage-3years-data-timeseries.zip'
    zip_ref = zipfile.ZipFile(file_name, 'r')
    zip_ref.extractall(path=data_path)
    zip_ref.close()

In [ ]:
if len(os.listdir()) > 1:
  power_usage = pd.read_csv('power_usage_2016_to_2020.csv')
  weather = pd.read_csv('weather_2016_2020_daily.csv')
if unzippingReq:
  StopWatch.stop("Data load")
  StopWatch.status("Data load", True)

## Data descriptive analysis

In [ ]:
power_usage.head(25)

In [ ]:
weather.head()

In [ ]:
print(power_usage.info())

In [ ]:
print(weather.info())

In [ ]:
power_usage.describe()

In [ ]:
power_usage.notes.unique()

## Data Preprocessing

### Change column name

In [ ]:
StopWatch.start("Data preprocessing")

In [ ]:
power_usage.rename(columns={'Value (kWh)' : 'Value'}, inplace=True)

### Change date format

The actual StartDate of data is from 2016-06-01(yyyy-mm-dd). In the dataset the dates are not properly formated. In the following steps the StartDate column is formated properly.

In [ ]:
m = power_usage.shape[0]
power_usage_date = pd.Series(range(m), pd.period_range('2016-06-01 00:00:00', freq = '1H', periods = m))
power_usage['StartDate'] = power_usage_date.to_frame().index

In [ ]:
m = weather.shape[0] 
weather_date = pd.Series(range(m), pd.period_range('2016-06-01', freq = '1D', periods = m))
weather['Date'] = weather_date.to_frame().index

### Merge datasets

For the purpose of merging Datasets we can split the StartDate to Date and Hour columns.

In [ ]:
for idx, date in power_usage.iterrows():
    power_usage.loc[idx, 'Date'] = power_usage.StartDate[idx].strftime('%Y-%m-%d')
    power_usage.loc[idx, 'Hour'] = power_usage.StartDate[idx].strftime('%H:%M')

power_usage.drop(['StartDate'], inplace=True, axis = 1)


weather_date = []
for idx, date in weather.iterrows():
    weather_date.append(weather.Date[idx].strftime('%Y-%m-%d'))
weather.Date = weather_date

In [ ]:
df_main  = power_usage.merge(weather,  on = ['Date', 'day_of_week'])

In [ ]:
StopWatch.stop("Data preprocessing")
StopWatch.status("Data preprocessing", True)

In [ ]:
print(f"Shape of main table is {df_main.shape}")
print(f"Shape of power_usage table is {power_usage.shape}")
print(f"Shape of weather table is {weather.shape}")

## Exploratory Data Analysis

In [ ]:
df_main.head()


For better use of data , we can split Date into year, month, day columns. Since day column is already in the dataframe, we might just add year and month column.

In [ ]:
df_main['year'] = df_main.Date.str.split('-', expand=True)[0]
df_main['month'] = df_main.Date.str.split('-', expand=True)[1]
df_main.drop(columns = ['Date'], inplace = True)

In [ ]:
tod = sns.lineplot('notes', 'Value', data=df_main)
plt.xlabel('Type of day')
plt.ylabel('Avg Value(kWh)')
plt.title('Average power usage by type of the day')
#tod.get_figure().savefig("tod.png")

In [ ]:
dow = sns.barplot('day_of_week', 'Value', data=df_main)
dow.set_xticklabels(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
plt.xlabel('Day of Week')
plt.ylabel('Avg Value(kWh)')
plt.title('Average power usage by day of the week')
plt.xticks(rotation = 90)
#dow.get_figure().savefig("dow.png")

In [ ]:
sns.lineplot(x = 'month', y = 'Value', data=df_main[df_main.year.astype(int) >= 2018], hue = 'year', ci = None)
plt.legend(labels = ['2018', '2019', '2020'])
plt.title('Monthly power usage of three years')
plt.ylabel('Power usage(kWh)')
#plt.savefig("monthly_power.png")

In [ ]:
target_corr = df_main.corr()

In [ ]:
sns.heatmap(target_corr, cmap='coolwarm')
plt.title('Correlation matrix')
#plt.savefig("corr_plot", pad_inches=0.6)

We can see that features like temperature, dew and pressure has high correlation to our target feature. Also different temperatures and dew features are inter-correlated. Therefore, all the intercorrelated features except for temp_avg can be dropped during feature selection. 

In [ ]:
target_corr['Value']

In [ ]:
drop_cols = ['Temp_max', 'Temp_min', 'Dew_avg', 'Dew_min', 'Dew_max', 'Press_avg', 'Press_min']

## Data Modelling

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# Pipelines
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import make_pipeline, Pipeline, FeatureUnion
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder

# CV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit

# metrics

from sklearn.metrics import mean_squared_error, r2_score

### Split data

In [ ]:
X_train, X_test,y_train , y_test = train_test_split(df_main.drop(columns=['Value']), df_main.Value, test_size = .2, stratify= df_main.notes, random_state =42, shuffle=True)

In [ ]:
print(f"Train data shape is {X_train.shape}")
print(f"Test data shape is {X_test.shape}")

In [ ]:
df_main.columns

### Pipelines

In [ ]:
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

In [ ]:
cat_features = ['day_of_week', 'notes', 'Hour', 'Day', 'year', 'month']
num_features = ['Temp_max', 'Temp_avg',
       'Temp_min', 'Dew_max', 'Dew_avg', 'Dew_min', 'Hum_max', 'Hum_avg',
       'Hum_min', 'Wind_max', 'Wind_avg', 'Wind_min', 'Press_max', 'Press_avg',
       'Press_min', 'Precipit']

In [ ]:
num_pipeline = Pipeline([
        ('selector', DataFrameSelector(num_features)),
        ('std_scaler', StandardScaler()),
    ])

# Create a pipelne for the categorical features.
# Entries with missing values or values that don't exist in the range
# defined above will be one hot encoded as zeroes.

cat_pipeline = Pipeline([
        ('selector', DataFrameSelector(cat_features)),
        ('ohe', OneHotEncoder(sparse=False, handle_unknown="ignore"))
    ])

# Union the transformed, scaled numeric and categorical features.
data_prep_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("cat_pipeline", cat_pipeline),
    ])

In [ ]:
lin_reg = LinearRegression()
np.random.seed(42)

baseline_pipeline = Pipeline([
    ('preperation', data_prep_pipeline),
    ('baseline_linreg', lin_reg)
])

In [ ]:
cvSplits = ShuffleSplit(n_splits=10, test_size=0.3, random_state=0)

### Baseline linear regression model

In [ ]:
StopWatch.start("Baseline Linear Regression")
baseline_pipeline.fit(X_train, y_train)
np.random.seed(42)

lin_scores = cross_val_score(baseline_pipeline, X_train, y_train, cv=cvSplits, scoring = 'neg_mean_squared_error')
                               
lin_score_train = np.sqrt(-lin_scores.mean())

# Time and score test predictions

lin_test_pred = baseline_pipeline.predict(X_test)

lin_score_test  = np.sqrt(mean_squared_error(y_test, lin_test_pred))
lin_r2_score = r2_score(y_test, lin_test_pred)

StopWatch.stop("Baseline Linear Regression")
StopWatch.status("Baseline Linear Regression", True)

In [ ]:
results = pd.DataFrame(columns=["ExpID", 
              "Train RMSE","Test RMSE", "Test R2"])

In [ ]:
results.loc[0] = ["Baseline Linear Regression", 
                  np.round(lin_score_train,4), np.round(lin_score_test,4), np.round(lin_r2_score, 4)]

pd.set_option('display.max_colwidth', None)
results

### Linear regression with predictors

In [ ]:
classifiers = [
        ('Linear Regression', LinearRegression()),
        ('Gradient Boosting', GradientBoostingRegressor(warm_start=True, random_state=42)),
        ('XGBoost', XGBRegressor(random_state=42)),
        ('Light GBM', LGBMRegressor(random_state=42)),
    ]

In [ ]:
#GradientBoostingRegressor().get_params().keys()

In [ ]:
params_grid = {
        'Linear Regression': {
            'fit_intercept': [True, False],
        },
        'Gradient Boosting':  {
            'max_depth': [10, 20, 30],
            'max_features': [20, 50, 100],
            'validation_fraction': [0.2],
            'n_iter_no_change': [10],
            'tol': [0.01],
            'n_estimators':[20, 50, 100],
        },
        'XGBoost':  {
            'max_depth': [10, 20, 30],
            'n_estimators':[20, 50, 100],
            'learning_rate': [0.1, 0.01, 0.001]
        },
        'Light GBM':  {
            'max_depth': [10, 20, 30],
            'num_leaves': [5, 10, 15],
            'n_estimators':[20, 50, 100],
            'learning_rate': [0.1, 0.01, 0.001],
            'reg_alpha': [0.1, 0.01, 0.001],
            'reg_lambda': [0.1, 0.01, 0.001],
        },
    }

#### Feature selection

In [ ]:
dropped_df = df_main.drop(columns=drop_cols).copy()
num_features = list(set(num_features) - set(drop_cols))
X_train, X_test,y_train , y_test = train_test_split(dropped_df.drop(columns=['Value']), dropped_df.Value, test_size = .2, stratify= dropped_df.notes, random_state =42, shuffle=True)

In [ ]:
num_pipeline = Pipeline([
        ('selector', DataFrameSelector(num_features)),
        ('std_scaler', StandardScaler()),
    ])

# Create a pipelne for the categorical features.
# Entries with missing values or values that don't exist in the range
# defined above will be one hot encoded as zeroes.

cat_pipeline = Pipeline([
        ('selector', DataFrameSelector(cat_features)),
      ('ohe', OneHotEncoder(sparse=False, handle_unknown="ignore"))
    ])

# Union the transformed, scaled numeric and categorical features.
data_prep_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("cat_pipeline", cat_pipeline),
    ])

In [ ]:
i = 1
for (name, classifier) in classifiers:
  StopWatch.start(name)
  parameters = params_grid[name].copy()

  params = {}
  for p in parameters.keys():
      pipe_key = 'predictor__'+str(p)
      params[pipe_key] = parameters[p] 

  pipe = Pipeline([('preperation', data_prep_pipeline),
    ('predictor', classifier)])
  
  print(f"********** Starting {name} modelling********")

  gridsearch_model = GridSearchCV(pipe, param_grid=params, cv = 5, n_jobs=-1, verbose=2)
  gridsearch_model.fit(X_train, y_train)

  best_scores = cross_val_score(gridsearch_model.best_estimator_, X_train, y_train, cv=cvSplits, scoring = 'neg_mean_squared_error')
                               
  best_score_train = -best_scores.mean()

  y_test_pred = gridsearch_model.best_estimator_.predict(X_test)

  best_score_test = mean_squared_error(y_test, y_test_pred)

  best_r2_test = r2_score(y_test, y_test_pred)

  StopWatch.stop(name)
  StopWatch.status(name, True)

  print(f"********** End of {name} modelling********")
  #best_scores_test  = baseline_pipeline.score(X_test, y_test)

  results.loc[i] = [name, 
                  np.round(best_score_train,4), np.round(best_score_test,4), np.round(best_r2_test, 4)]

  pd.set_option('display.max_colwidth', None)

  i += 1
  

In [ ]:
results

In [ ]:
StopWatch.benchmark()